In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Loading data
df_poi = pd.read_csv(r'granular_pois.csv')
df_barnet = pd.read_csv(r'barnet.csv')
lsoa_boundaries=gpd.read_file('barnet.geojson')

# South Barnet LSOA list
select = ['E01000221', 'E01000223', 'E01000141', 'E01000184', 'E01000178', 'E01000176'] + list(df_barnet[(df_barnet['LSOA name'].str.contains('038')) | (df_barnet['LSOA name'].str.contains('029')) | (df_barnet['LSOA name'].str.contains('033')) | (df_barnet['LSOA name'].str.contains('035')) | (df_barnet['LSOA name'].str.contains('037')) | (df_barnet['LSOA name'].str.contains('040')) | (df_barnet['LSOA name'].str.contains('041'))]['lsoa11cd'].unique())

In [ ]:
# Get POI per LSOA
geometry = [Point(xy) for xy in zip(df_poi['long'], df_poi['lat'])]
gdf_poi = gpd.GeoDataFrame(df_poi, geometry=geometry)

poi_per_lsoa = gpd.sjoin(lsoa_boundaries, gdf_poi, how='left', op='contains')

In [ ]:
significant_lsoa = ['E01000138', 'E01000139', 'E01000145', 'E01000146', 'E01000211']
df_poi_south = poi_per_lsoa[poi_per_lsoa['lsoa11cd'].isin(select)]
categories = list(df_poi_south['POI'].unique())
selected_facilities = ['gym', 'park', 'groceries', 'cafe']

In [ ]:
display_count = 'Crime Count'

df_series = df_barnet.groupby(['lsoa11cd', 'year'])['Crime ID'].count().reset_index(name='Crime Count')

lsoa_crime_counts = df_series[df_series['lsoa11cd'].isin(select)]

lsoa_chosen = select
option_slctd = 2017
lsoa_crime_counts = lsoa_crime_counts.groupby(['lsoa11cd', 'year']).sum().reset_index()
lsoa_crime_counts = lsoa_crime_counts[lsoa_crime_counts['year'] == option_slctd]

lsoa_stats = lsoa_crime_counts[display_count].describe()

print(lsoa_stats)

m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)
folium.Choropleth(
    geo_data=lsoa_boundaries,
    name='choropleth',
    data=lsoa_crime_counts,
    columns=['lsoa11cd', display_count],
    key_on='feature.properties.lsoa11cd',
    fill_color='YlOrRd',
    fill_opacity=0.7,  
    line_opacity=0.2,
    legend_name=f'{display_count} in {option_slctd}',
    highlight=True,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['lsoa11cd', display_count],
        aliases=['LSOA Code', display_count],
        sticky=True,
        labels=True,
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    ),
    #  threshold_scale=custom_scale
).add_to(m)

for index, row in df_poi_south[df_poi_south['POI'].isin(selected_facilities)].iterrows():
    stop_lat = row['lat']
    stop_lon = row['long'] 

    marker = folium.CircleMarker(location=[stop_lat, stop_lon], radius=4, color='black', fill=True, fill_color='black', popup = row['POI'])
    marker.add_to(m)
m